In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras import datasets, layers, models
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import ReLU, concatenate
mx_len=2584
import csv 

In [2]:
path='/content/drive/MyDrive/assignment 1/annotations.csv'
relative_path='/content/drive/MyDrive/assignment 1/train/train/'

In [3]:
def load_data(path,relative_path):
    df=pd.read_csv(path)
    data=[]
    for filename in df['fname']:
        address=relative_path+filename
        data.append(np.load(address))
    label=df['label']
    
    return data,label

In [4]:
data,label=load_data(path,relative_path)


In [5]:
def list_to_np(data,max_len):
    final_data=[]
    for spectrogram in data:
        x=spectrogram.flatten()
        y=np.zeros((max_len-spectrogram.shape[2])*64)
        y=np.append(y,x)
        z=np.zeros((max_len-spectrogram.shape[2])*64)
        y=np.append(y,z)
        final_data.append(y)
    return np.array(final_data)


In [6]:
data=list_to_np(data,mx_len)


In [7]:
def mfcc(data,mean=1):
  k=0
  if mean!=0:
    p=np.zeros((len(data),128))
  else:
    p=np.zeros((len(data),128,647))
  for i in range(len(data)):
    mfcc_feature=librosa.feature.mfcc(y=data[i],sr=44100,n_fft=2048,hop_length=512,n_mfcc=128,window='hann')  
    k=max(k,np.max(mfcc_feature))
    
    if mean!=0:
      mfcc_feature=np.mean(mfcc_feature,axis=1)    
      mfcc_feature=mfcc_feature.flatten()
      mfcc_feature=np.array(mfcc_feature)      
    p[i]=mfcc_feature
  p=p/k
  return p

In [8]:
mfcc_data=mfcc(data,0)


In [9]:
d=[]
for i in range(len(data)):
  k=mfcc_data[i].flatten()
  k=np.reshape(k,(128,647,1))
  
  d.append(k)
d=np.array(d)


In [10]:
label=pd.get_dummies(label)

In [ ]:
train_x,test_x,train_y,test_y=train_test_split(d,label,test_size=0.2,random_state=9)

In [ ]:
model = Sequential()
model.add(layers.Conv2D(128, (2, 2), activation='relu', input_shape=(128,647,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (2, 2), activation='relu'))

model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_x, train_y, epochs=20)

In [ ]:
model.evaluate(test_x,test_y)

7/7 [==============================] - 13s 2s/step - loss: 2.7348 - accuracy: 0.7500


[2.73478364944458, 0.75]

In [ ]:
### FINAL MODEL

In [11]:
final_model=Sequential()
final_model.add(layers.Conv2D(128, (2, 2), activation='relu', input_shape=(128,647,1)))
final_model.add(layers.MaxPooling2D((2, 2)))
final_model.add(layers.Dropout(0.2))
final_model.add(layers.Conv2D(64, (2, 2), activation='relu'))

final_model.add(layers.Conv2D(64, (2, 2), activation='relu'))
final_model.add(layers.MaxPooling2D((2, 2)))
final_model.add(layers.Dropout(0.2))
final_model.add(layers.Flatten())
final_model.add(layers.Dense(64, activation='relu'))
final_model.add(layers.Dense(10, activation='softmax'))
final_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
history = final_model.fit(d,label, epochs=30)

Epoch 1/30
32/32 [==============================] - 18s 243ms/step - loss: 2.2617 - accuracy: 0.1500
Epoch 2/30
32/32 [==============================] - 7s 232ms/step - loss: 1.8933 - accuracy: 0.3560
Epoch 3/30
32/32 [==============================] - 8s 238ms/step - loss: 1.3969 - accuracy: 0.5440
Epoch 4/30
32/32 [==============================] - 7s 232ms/step - loss: 1.0054 - accuracy: 0.6650
Epoch 5/30
32/32 [==============================] - 7s 232ms/step - loss: 0.7408 - accuracy: 0.7670
Epoch 6/30
32/32 [==============================] - 7s 232ms/step - loss: 0.5031 - accuracy: 0.8330
Epoch 7/30
32/32 [==============================] - 7s 232ms/step - loss: 0.4481 - accuracy: 0.8460
Epoch 8/30
32/32 [==============================] - 7s 234ms/step - loss: 0.2495 - accuracy: 0.9230
Epoch 9/30
32/32 [==============================] - 7s 234ms/step - loss: 0.2036 - accuracy: 0.9330
Epoch 10/30
32/32 [==============================] - 7s 234ms/step - loss: 0.0938 - accuracy: 0.977

In [13]:
p='/content/drive/MyDrive/assignment 1/test_1/'

In [94]:

test_data=[]
test_name=[]
for filename in os.listdir(p):
    if filename.endswith('.npy'):
      test_name.append(filename)
      filename=p+filename
      test_data.append(np.load(filename))


In [95]:
test_data=list_to_np(test_data,mx_len)

In [96]:
test_data=mfcc(test_data,0)

TypeError: ignored

In [97]:
d=[]
for i in range(len(test_data)):
  k=mfcc_data[i].flatten()
  k=np.reshape(k,(128,647,1))
  
  d.append(k)
d=np.array(d)

In [98]:
y_pred=final_model.predict(d)
max_list = []
max = 0
for i in range(201):
  max_list.append(np.max(y_pred[i]))
index_list = []
for i in range(201):
  for j in range(10):
    if y_pred[i][j] == max_list[i]:
      index_list.append(j)
arr = []
z=['Bark','Meow','Siren','Shatter','Knock','Crying_and_sobbing','Microwave_oven','Vehicle_horn_and_car_horn_and_honking','Doorbell','Walk_and_footsteps',]
for i in range(201):
  arr.append(z[index_list[i]])
test_name,arr

(['MLSP_test1216256.npy',
  'MLSP_test1134284.npy',
  'MLSP_test1205141.npy',
  'MLSP_test1163322.npy',
  'MLSP_test1077269.npy',
  'MLSP_test1053175.npy',
  'MLSP_test1071.npy',
  'MLSP_test1194176.npy',
  'MLSP_test1192345.npy',
  'MLSP_test113179.npy',
  'MLSP_test1186185.npy',
  'MLSP_test1140162.npy',
  'MLSP_test1116162.npy',
  'MLSP_test130845.npy',
  'MLSP_test1458291.npy',
  'MLSP_test1457299.npy',
  'MLSP_test1395138.npy',
  'MLSP_test1328457.npy',
  'MLSP_test1277465.npy',
  'MLSP_test1379148.npy',
  'MLSP_test1362223.npy',
  'MLSP_test1373407.npy',
  'MLSP_test1404304.npy',
  'MLSP_test1289164.npy',
  'MLSP_test1306499.npy',
  'MLSP_test1357311.npy',
  'MLSP_test140846.npy',
  'MLSP_test132489.npy',
  'MLSP_test130220.npy',
  'MLSP_test1443477.npy',
  'MLSP_test1353495.npy',
  'MLSP_test1451403.npy',
  'MLSP_test1235269.npy',
  'MLSP_test1301172.npy',
  'MLSP_test1475459.npy',
  'MLSP_test1602453.npy',
  'MLSP_test1778475.npy',
  'MLSP_test1706153.npy',
  'MLSP_test1798478.

In [99]:

arr

['Bark',
 'Bark',
 'Shatter',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Meow',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Meow',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Bark',
 'Knock',
 'Knock',
 'Meow',
 'Knock',
 'Knock',
 'Knock',
 'Vehicle_horn_and_car_horn_and_honkin

In [100]:

df = pd.DataFrame(list(zip(test_name, arr)),columns =['filename', 'predictions'])

In [101]:
df

,filename,predictions
0,MLSP_test1216256.npy,Bark
1,MLSP_test1134284.npy,Bark
2,MLSP_test1205141.npy,Shatter
3,MLSP_test1163322.npy,Bark
4,MLSP_test1077269.npy,Bark
...,...,...
196,MLSP_test774107.npy,Knock
197,MLSP_test8395.npy,Knock
198,MLSP_test755223.npy,Knock
199,MLSP_test997159.npy,Knock


In [102]:
from google.colab import files

df.to_csv('201156.csv', encoding = 'utf-8-sig') 
files.download('201156.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
lo